### **Importing Libraries**

In [2]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR, CyclicLR, OneCycleLR, StepLR

import numpy as np
import random

from datetime import datetime
from pathlib import Path
import sys
from types import SimpleNamespace

from IPython.display import Image
import matplotlib.pyplot as plt
%matplotlib inline

import wandb

### **Set seed**

In [2]:
SEED = 2345
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### **Wanbd Login**

In [3]:
# Login to W&B
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: piyushrs. Use `wandb login --relogin` to force relogin


True

### **Data Folder declaration**

In [3]:
base_folder = Path('u:\\OneDrive - The University of Texas at Dallas\\6382\\Datasets')

In [4]:
data_folder = base_folder/'data'
archive_folder = base_folder/'archive'
model_folder = base_folder/'models/CIFAR_10'
custom_functions = base_folder/'custom-functions'

In [5]:
model_folder.mkdir(exist_ok=True, parents=True)
data_folder.mkdir(exist_ok=True, parents=True)

In [7]:
sys.path.append(str(custom_functions))
sys.path

['u:\\OneDrive - The University of Texas at Dallas\\6382\\Assignment 6',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310',
 '',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\piyus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\Pythonwin',
 'u:\\OneDrive - The University of Texas at Dallas\\6382\\Datasets\\custom-functions']

### **Transformations and downloading dataset**

In [6]:
trans1 = transforms.ToTensor()
trans2 = transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
trans = transforms.Compose([trans1, trans2])

train_val_set = torchvision.datasets.CIFAR10(root = data_folder,  train = True, transform = trans, download = True)

testset = torchvision.datasets.CIFAR10(root = data_folder, train = False, transform = trans, download = True)

Files already downloaded and verified
Files already downloaded and verified


### **Split dataset function**

In [7]:
def split_dataset(base_dataset, fraction, seed):
    split_a_size = int(fraction * len(base_dataset))
    split_b_size = len(base_dataset) - split_a_size
    return torch.utils.data.random_split(base_dataset, [split_a_size, split_b_size], generator=torch.Generator().manual_seed(seed)
    )

In [8]:
trainset, validset = split_dataset(train_val_set, 0.8, 42)

In [9]:
len(trainset), len(validset)

(40000, 10000)

In [10]:
trainset.dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [11]:
trainset.dataset.data.shape

(50000, 32, 32, 3)

### <font color = "teal"> **Custom CNN Module**

In [14]:
class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(5,5), padding=2, stride = 1, bias=True),      # 32 x 32 x 32
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels= 32, out_channels=16, kernel_size=(3,3), padding = 1, bias = True),        #  32 x 32 x 16
            nn.ReLU(inplace=True)
        )
        self.classifier = nn.Sequential(
            nn.Linear(32*32*16, 10, bias=True)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits

### <font color = "aqua"> **CustomCNN torch summary**

In [15]:
from torchsummary import summary
summary(CustomCNN().cuda(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]           2,432
              ReLU-2           [-1, 32, 32, 32]               0
            Conv2d-3           [-1, 16, 32, 32]           4,624
              ReLU-4           [-1, 16, 32, 32]               0
            Linear-5                   [-1, 10]         163,850
Total params: 170,906
Trainable params: 170,906
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.75
Params size (MB): 0.65
Estimated Total Size (MB): 1.41
----------------------------------------------------------------


### **Train function**

In [17]:
def train(train_loader, loss_function, model, optimizer, device, grad_clipping, max_norm, log_batch, log_interval):

  # Training Loop 

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_train

  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode

  model.train()
  # Iterate on batches from the dataset using train_loader
  for input_, targets in train_loader:
    
    # move inputs and outputs to GPUs
    input_ = input_.to(device)
    targets = targets.to(device)


    # Step 1: Forward Pass: Compute model's predictions 
    output = model(input_)
    
    # Step 2: Compute loss
    loss = loss_function(output, targets)

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    correct = torch.sum(y_pred == targets)

    batch_ct_train += 1

    # Step 3: Backward pass -Compute the gradients
    optimizer.zero_grad()
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Step 4: Update the parameters
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        wandb.log({f"Train Batch Acc :": correct/len(targets)})

  
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)

  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)
  

  return train_loss, train_acc

### **Validate Function**

In [18]:
def validate(valid_loader, loss_function, model, device, log_batch, log_interval):

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_valid

  # Validation/Test loop
  # Initialize valid_loss at the he strat of the epoch
  running_val_loss = 0
  running_val_correct = 0

  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for input_,targets in valid_loader:

      # move inputs and outputs to GPUs
      input_ = input_.to(device)
      targets = targets.to(device)

      # Step 1: Forward Pass: Compute model's predictions 
      output = model(input_)

      # Step 2: Compute loss
      loss = loss_function(output, targets)

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      correct = torch.sum(y_pred == targets)

      batch_ct_valid += 1

      # Add val loss of a batch 
      running_val_loss += loss.item()

      # Add correct count for each batch
      running_val_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          wandb.log({f"Valid Batch Accuracy :": correct/len(targets)})

    # Calculate mean val loss for the whole dataset for a particular epoch
    val_loss = running_val_loss/len(valid_loader)

    # Calculate accuracy for the whole dataset for a particular epoch
    val_acc = running_val_correct/len(valid_loader.dataset)

    # scheduler step
    # scheduler.step(val_loss)
    # scheduler.step()
    
  return val_loss, val_acc

### **Train Loop Function**

In [19]:
def train_loop(train_loader, valid_loader, model, optimizer, loss_function, epochs, device, patience, early_stopping,
               file_model, save_best_model):
    
  """ 
  Function for training the model and plotting the graph for train & validation loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and validation loss for each epoch.
  """

  # Create lists to store train and val loss at each epoch
  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []

  # initialize variables for early stopping

  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False

  # Iterate for the given number of epochs
  # Step 5: Repeat steps 1 - 4

  for epoch in range(epochs):

    t0 = datetime.now()

    # Get train loss and accuracy for one epoch
    train_loss, train_acc = train(train_loader, loss_function, model, optimizer,
                                  wandb.config.device, wandb.config.grad_clipping, 
                                  wandb.config.max_norm, wandb.config.log_batch, wandb.config.log_interval)
    valid_loss, valid_acc   = validate(valid_loader, loss_function, model, wandb.config.device, wandb.config.log_batch, wandb.config.log_interval)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)

    valid_loss_history.append(valid_loss)
    valid_acc_history.append(valid_acc)

    # Log the train and valid loss to wandb
    wandb.log({f"Train Loss :": train_loss, "epoch": epoch})
    wandb.log({f"Train Acc :": train_acc, "epoch": epoch})

    wandb.log({f"Valid Loss :": valid_loss, "epoch": epoch})
    wandb.log({f"Valid Acc :": valid_acc, "epoch": epoch})

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True

      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    elif save_best_model:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss
        
    else:
        torch.save(model.state_dict(), file_model)
    
    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    # print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: {valid_acc * 100 : .4f}%')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history

### **Get accuracy prediction function**

In [20]:
def get_acc_pred(data_loader, model, device):
    
  """ 
  Function to get predictions and accuracy for a given data using estimated model
  Input: Data iterator, Final estimated weoights, bias
  Output: Prections and Accuracy for given dataset
  """

  # Array to store predicted labels
  predictions = torch.Tensor() # empty tensor
  predictions = predictions.to(device) # move predictions to GPU

  # Array to store actual labels
  y = torch.Tensor() # empty tensor
  y = y.to(device)

  # put the model in evaluation mode
  model.eval()
  
  # Iterate over batches from data iterator
  with torch.no_grad():
    for input_, targets in data_loader:
      
      # move inputs and outputs to GPUs
      
      input_ = input_.to(device)
      targets = targets.to(device)
      
      # Calculated the predicted labels
      output = model(input_)

      # Choose the label with maximum probability
      prediction = torch.argmax(output, dim = 1)

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, prediction)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

  # Check for complete dataset if actual and predicted labels are same or not
  # Calculate accuracy
  acc = (predictions == y).float().mean()

  # Return tuple containing predictions and accuracy
  return predictions, acc  

### **Meta Data** (overfitting smaller subset of trainset)

In [21]:
hyperparameters = SimpleNamespace(
    epochs = 10,
    output_dim = 10,
    batch_size= 50,
    learning_rate=0.01,
    dataset="CIFAR10",
    architecture="CustomCNN",
    log_interval = 25,
    log_batch = True,
    file_model = model_folder/'assignment_customNN_CIFAR10.pt',
    grad_clipping = False,
    max_norm = 0,
    momentum = 0.9,
    patience = 0,
    early_stopping = False,
    scheduler_factor = 0,
    scheduler_patience = 0,
    weight_decay = 0,
    save_best_model = True,
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

In [23]:
import torch
torch.cuda.get_device_properties("cuda:0")

_CudaDeviceProperties(name='NVIDIA GeForce RTX 2060', major=7, minor=5, total_memory=6143MB, multi_processor_count=30)

In [22]:
wandb.init(name = "exp1", project = 'CustomCNN_HW6' , config = hyperparameters)

In [23]:
wandb.config

{'epochs': 10, 'output_dim': 10, 'batch_size': 50, 'learning_rate': 0.01, 'dataset': 'CIFAR10', 'architecture': 'CustomCNN', 'log_interval': 25, 'log_batch': True, 'file_model': 'u:\\OneDrive - The University of Texas at Dallas\\6382\\Datasets\\models\\CIFAR_10\\assignment_customNN_CIFAR10.pt', 'grad_clipping': False, 'max_norm': 0, 'momentum': 0.9, 'patience': 0, 'early_stopping': False, 'scheduler_factor': 0, 'scheduler_patience': 0, 'weight_decay': 0, 'save_best_model': True, 'device': 'cuda:0'}

### **Data Loaders**

In [24]:
subset_size = int(len(trainset)/400)
subset_indices = random.sample(range(0, len(trainset)), subset_size)
subset_sample = torch.utils.data.Subset(trainset, subset_indices)
trainsubset = torch.utils.data.DataLoader(subset_sample, batch_size= wandb.config.batch_size, shuffle= False)

valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.batch_size, shuffle=False)

### **Init weights with Kaiming initialization**

In [25]:
def init_weight(layer):
  if type(layer) == nn.Linear:
    torch.nn.init.kaiming_normal_(layer.weight)
    torch.nn.init.zeros_(layer.bias)
  
  if type(layer) == nn.Conv2d:
    torch.nn.init.kaiming_normal_(layer.weight)
    torch.nn.init.zeros_(layer.bias)

In [26]:
model = CustomCNN()
model.to(wandb.config.device)
model.apply(init_weight)
loss_function = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr= wandb.config.learning_rate, 
                                            momentum=wandb.config.momentum, nesterov= True)

### <font color = "Red"> **Training Model**

In [27]:
wandb.watch(model)

[]

In [28]:
# See live graphs in the notebook.
#%%wandb 
batch_ct_train, batch_ct_valid = 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(trainsubset, 
                                                                                          valid_loader, 
                                                                                          model, 
                                                                                          optim, 
                                                                                          loss_function, 
                                                                                          wandb.config.epochs, 
                                                                                          wandb.config.device,
                                                                                          wandb.config.patience,
                                                                                          wandb.config.early_stopping,
                                                                                          wandb.config.file_model,
                                                                                          wandb.config.save_best_model
                                                                                          )

Validation loss has decreased (inf --> 2.834355). Saving Model...
Epoch : 1 / 10
Time to complete 1 is 0:00:02.253217
Train Loss:  3.3098 | Train Accuracy:  10.0000%
Valid Loss:  2.8344 | Valid Accuracy:  14.1200%

Validation loss has decreased (2.834355 --> 2.274717). Saving model...
Epoch : 2 / 10
Time to complete 2 is 0:00:02.198299
Train Loss:  2.5384 | Train Accuracy:  24.0000%
Valid Loss:  2.2747 | Valid Accuracy:  14.8900%

Validation loss has decreased (2.274717 --> 2.246001). Saving model...
Epoch : 3 / 10
Time to complete 3 is 0:00:02.225541
Train Loss:  2.0079 | Train Accuracy:  46.0000%
Valid Loss:  2.2460 | Valid Accuracy:  14.7400%

Validation loss has not decreased (2.246001 --> 2.422808). Not Saving Model...
Epoch : 4 / 10
Time to complete 4 is 0:00:02.592068
Train Loss:  1.7431 | Train Accuracy:  46.0000%
Valid Loss:  2.4228 | Valid Accuracy:  15.6700%

Validation loss has not decreased (2.246001 --> 2.449852). Not Saving Model...
Epoch : 5 / 10
Time to complete 5 is 0

In [29]:
wandb.finish()

Train Acc :,▁▂▄▄▄▅▆▇██
Train Loss :,█▆▅▄▄▃▃▂▁▁
Valid Acc :,▁▂▂▃▆██▇▆▇
Valid Batch Accuracy :,▄▃▆▃█▃▅▃▅▁▃▃▁▃▄▅▄▃▄▆▄▃▆▇▅▆▆▇▃▆▅▅▅▄▅▅▄▆▄█
Valid Batch Loss :,▃▅▂▃▁▂▁▁▁▁▂▁▂▂▂▂▂▂▂▁▂▁▂▁▂▁▃▂▃▃▄▃▅▅▆▅▆▇▇█
Valid Loss :,▃▁▁▂▂▂▂▄▆█
epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
Train Acc :,0.92
Train Loss :,0.31587
Valid Acc :,0.1974
Valid Batch Accuracy :,0.24


## <font color = "aqua"> **Training Full Dataset**

In [30]:
hyperparameters = SimpleNamespace(
    epochs = 15,
    output_dim = 10,
    batch_size= 40,
    learning_rate=0.01,
    dataset="CIFAR10",
    architecture="CustomCNN",
    log_interval = 25,
    log_batch = True,
    file_model = model_folder/'assignment_customNN_CIFAR10.pt',
    grad_clipping = False,
    max_norm = 0,
    momentum = 0.9,
    patience = 0,
    early_stopping = False,
    scheduler_factor = 0,
    scheduler_patience = 0,
    weight_decay = 0,
    save_best_model = True,
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

In [31]:
wandb.init(name = "exp2", project = 'CustomCNN_HW6' , config = hyperparameters)

In [32]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size= wandb.config.batch_size, shuffle= True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.batch_size, shuffle=False)

In [33]:
model = CustomCNN()
model.to(wandb.config.device)
model.apply(init_weight)
loss_function = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr= wandb.config.learning_rate, 
                                            momentum=wandb.config.momentum, nesterov= True)

In [34]:
wandb.watch(model)

[]

In [35]:
# See live graphs in the notebook.
#%%wandb 
batch_ct_train, batch_ct_valid = 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, 
                                                                                          valid_loader, 
                                                                                          model, 
                                                                                          optim, 
                                                                                          loss_function, 
                                                                                          wandb.config.epochs, 
                                                                                          wandb.config.device,
                                                                                          wandb.config.patience,
                                                                                          wandb.config.early_stopping,
                                                                                          wandb.config.file_model,
                                                                                          wandb.config.save_best_model
                                                                                          )

Validation loss has decreased (inf --> 1.219289). Saving Model...
Epoch : 1 / 15
Time to complete 1 is 0:00:14.318595
Train Loss:  1.4436 | Train Accuracy:  49.0800%
Valid Loss:  1.2193 | Valid Accuracy:  56.5000%

Validation loss has decreased (1.219289 --> 1.160565). Saving model...
Epoch : 2 / 15
Time to complete 2 is 0:00:13.943355
Train Loss:  1.1127 | Train Accuracy:  61.2925%
Valid Loss:  1.1606 | Valid Accuracy:  59.3700%

Validation loss has decreased (1.160565 --> 1.150188). Saving model...
Epoch : 3 / 15
Time to complete 3 is 0:00:14.000636
Train Loss:  0.9347 | Train Accuracy:  67.8050%
Valid Loss:  1.1502 | Valid Accuracy:  60.3000%

Validation loss has not decreased (1.150188 --> 1.225996). Not Saving Model...
Epoch : 4 / 15
Time to complete 4 is 0:00:13.962097
Train Loss:  0.7901 | Train Accuracy:  72.8275%
Valid Loss:  1.2260 | Valid Accuracy:  58.8900%

Validation loss has not decreased (1.150188 --> 1.325119). Not Saving Model...
Epoch : 5 / 15
Time to complete 5 is 0

In [36]:
wandb.finish()

Train Acc :,▁▃▄▅▅▆▇▇▇██████
Train Batch Acc :,▁▁▃▃▄▅▄▃▅▅▆▆▅▅▆▆▇▆▆▇▆▅▇▇█▆▆▇▇▇▇▇▇█▇▇▇█▇▇
Train Batch Loss :,█▇▆▇▄▄▅▆▄▄▃▂▄▄▂▄▂▃▄▂▂▃▂▂▁▃▂▂▂▁▂▂▁▁▂▁▂▁▂▁
Train Loss :,█▆▅▄▄▃▂▂▂▁▁▁▁▁▁
Valid Acc :,▄▇█▇▇▆▄▄▄▄▃▂▂▂▁
Valid Batch Accuracy :,▁▂█▅▅▅▄▆▅▅▇▅▄▅▄▅▅▄▄▃▄▅▄▇▄▃▃▅▅▄▂▄▃▃▂▄▅▇▄▃
Valid Batch Loss :,▂▂▁▁▁▂▁▁▁▁▁▁▂▂▂▂▂▂▃▂▃▃▃▂▃▃▅▃▄▃▅▄▄▆▇▄▄▃▆█
Valid Loss :,▁▁▁▁▂▂▂▃▄▅▅▆▆▇█
epoch,▁▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇██
Train Acc :,0.9339
Train Batch Acc :,0.925
